In [2]:
# for import & export excel file
from openpyxl import load_workbook, Workbook

# Install pymysql package if you haven't already
!pip install pymysql
import pymysql

# Establish a connection to the MySQL database
connection = pymysql.connect(
    host='localhost',
    port= 3306,
    user='root',
    password='xxxx', # censor my password
    database='tama_want_to_buy',
    cursorclass=pymysql.cursors.DictCursor
)

# Check if the connection is successful
if connection:
    print("Connection successful!")
else:
    print("Connection failed.")

Connection successful!


In [3]:
# Step01: Load new data from the Excel file named 'imported_02.xlsx'
workbook = load_workbook('imported_02.xlsx')  # Open the Excel workbook
sheet = workbook.active  # Get the active sheet within the workbook


# Create a cursor to execute SQL queries on the database
cursor = connection.cursor()

# Query all data from the 'categories' table in MySQL
sql_select_categories = '''
        SELECT * FROM categories;
'''
cursor.execute(sql_select_categories)  # Execute the query
categories = cursor.fetchall()  # Fetch all the results into 'categories'


# Step 02: Iterate over each row in the Excel sheet to compare categories with MySQL data
categories_values = []  # List to store new categories
for row in sheet.iter_rows(min_row=2, values_only=True):  # Skip headers, start from second row
    is_new = True
    category = row[3]  # Get category value from the row
    
    # Check if the category from the Excel sheet is already in MySQL
    for c in categories:
        if category == c['name']:  # Compare category names
            is_new = False  # Category already exists, mark as not new
            break
            
    if is_new:
        print((category, )) # Print what data will append into tuple
        categories_values.append((category,))  # Add new category to the list as a tuple

# Step 03: If new categories are found, insert them into the 'categories' table in MySQL
if len(categories_values) > 0:
    sql_insert_categories = '''
        INSERT INTO categories (name)
        VALUES (%s)
    '''
    cursor.executemany(sql_insert_categories, categories_values)  # Execute insertion query
    connection.commit()  # Commit the changes to the database

    # Print the status of the insertion process
    print('เพิ่มประเภทสินค้าจำนวน ' + str(cursor.rowcount) + ' แถว')
else:
    print('ไม่มีประเภทสินค้าใหม่ให้เพิ่ม')  # No new categories found

ไม่มีประเภทสินค้าใหม่ให้เพิ่ม


In [16]:
# Query all data from the 'categories' table in MySQL
cursor.execute(sql_select_categories)
categories = cursor.fetchall()  # Fetch all categories from MySQL

products_values = []  # List to store new products

# Iterate over each row in the Excel sheet to match category_id for inputting into new products
for row in sheet.iter_rows(min_row=2, values_only=True):  # Skip headers, start from the second row
    category_name = row[3]  # Get the category name from the Excel file
    category_id = 'None'  # Default category ID to None
    
    # Iterate through categories fetched from MySQL to find matching category name
    for category in categories:
        if category_name == category['name']:  # Compare category names
            category_id = category['id']  # If a match is found, assign the category ID
            break
    
    # Create a tuple representing the product data with its category ID
    product = (row[0], row[1], row[2], category_id)
    print(product)  # Print the product tuple (for debugging)
    products_values.append(product)  # Add the product tuple to the list of product values

# SQL query to insert new products into the 'products' table in MySQL
sql_insert_products = '''
    INSERT INTO products (name, price, is_necessary, category_id)
    VALUES (%s, %s, %s, %s);
'''

# Execute the insertion query for each product and commit the changes to the database
cursor.executemany(sql_insert_products, products_values)
connection.commit()

# Print the status of the insertion process
print('เพิ่มสิน้คาจำนวน ' + str(cursor.rowcount) + ' แถว')


('เมนบอร์ด', 6000, 0, 1)
('หมวกลูฟี่', 350, 0, 10)
('โคมไฟตั้งพื้น', 2500, 1, 11)
เพิ่มสิน้คาจำนวน 3 แถว


In [12]:
# then close the coursor and connection with MySQL
cursor.close()
connection.close()